In [3]:
%run 'ks29_Get_folder_structure.ipynb'

### Reading weights

In [5]:
from collections import OrderedDict
from IPython.display import display
import pandas as pd

In [6]:
# Relative >>> absolute position
position_translation = pd.read_table(files_dump_folder + 'position_translation.csv')
position_translation.set_index('relative_position', inplace=True)

def get_absolute_position(segment_short, relative_position):
    return int(position_translation.iloc[int(relative_position)][segment_short])

In [18]:
Scer_Uniprot = open(os.path.join(files_dump_folder, 'HIS3_saccharomyces_cerevisiae_from_Uniprot_P06633.txt')).read().rstrip()

def get_mutant_weight(segment, genotype):
    mutant_weight = 0
    subset = weights_dict[segment]
#     display(subset)
    mutated_positions = [int(m[:-1]) for m in genotype.split(':')]
    for position in subset.position.values:
        if position not in mutated_positions:
            wt_aa = Scer_Uniprot[position-1]
            wt_state = str(position) + wt_aa
            wt_weight = subset.loc[wt_state]['weight']
            mutant_weight += wt_weight
    for mutation in genotype.split(':'):
        mutant_weight += subset.loc[mutation]['weight']
    return mutant_weight

In [19]:
weights_folder = os.path.join(analysis_folder, 'Katya', 'NN', 'weights', '')
weights_dict = OrderedDict()
for segment in ['S%s' %i for i in range(1,13)]:
    weights_dict[segment] = pd.read_csv(weights_folder + '%s.csv' %segment)
    weights_dict[segment]['relative_numbering'] = weights_dict[segment]['Unnamed: 0']
    weights_dict[segment]['mut_list_Scer'] = weights_dict[segment]['Unnamed: 0'].apply(
        lambda s: str(get_absolute_position(segment, int(s[:-1]))) + s[-1])
    weights_dict[segment]['segment'] = segment + '_scaled'
    weights_dict[segment]['position'] = weights_dict[segment]['mut_list_Scer'].apply(lambda m: int(m[:-1]))
    weights_dict[segment].set_index('mut_list_Scer', inplace=True)
    weights_dict[segment]['mutant_weight'] = weights_dict[segment].index.map(lambda m: get_mutant_weight(segment, m))

In [29]:
def get_full_mutation(mutation):
    if mutation == '' or not mutation > 0:
        return ''
    position = int(mutation[:-1])
    return Scer_Uniprot[position-1] + mutation

def convert_to_full_mutations(mut_combination):
    if mut_combination == '' or not mut_combination > 0:
        return ''
    return ':'.join(get_full_mutation(mutation) for mutation in mut_combination.split(':'))

In [32]:
weights = pd.concat(weights_dict.values(), axis=0)
weights['mut_list_Scer_full'] = weights.index.map(convert_to_full_mutations)
weights.drop('Unnamed: 0', inplace=True, axis=1)

In [91]:
print '''
Variables: 

- weights: pandas.DataFrame containing weights of mutations from neural network data (column 'weight') and weights, or fitness potential, of single mutants (column 'mutant_weight')
'''


Variables: 

- weights: pandas.DataFrame containing weights of mutations from neural network data (column 'weight') and weights, or fitness potential, of single mutants (column 'mutant_weight')

